In [5]:
# HEAVILY DRAWN FROM: 
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
import pandas as pd
import tensorflow as tf
import os
BATCH_SIZE = 2

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU:0')))

Num GPUs Available:  0


In [7]:
training_path = os.path.join("./", "final_results/", "trainment.csv")
df = pd.read_csv(training_path)
df.tail()

,affected_day,ticker,polarity_title,subjectivity_title,polarity_description,subjectivity_description,Open,Close,Open-Close,target
460,2023-05-08,NVDA,-1.586033e-17,0.164286,0.135958,0.484572,285.220001,291.510010,-6.290009,0
461,2023-05-08,PYPL,0.000000e+00,0.000000,0.172619,0.586905,75.709999,75.519997,0.190002,1
462,2023-05-08,TSLA,9.444444e-02,0.155556,0.173279,0.439407,173.720001,171.789993,1.930008,1
463,2023-05-08,TTD,0.000000e+00,0.000000,0.308333,0.558333,62.549999,64.669998,-2.119999,0
464,2023-05-08,ZG,3.321429e-01,0.703571,0.092092,0.341407,47.500000,47.380001,0.119999,1


In [8]:
target = df.pop('target')

In [9]:
numeric_feature_names = [
    'polarity_title', 'subjectivity_title',
    'polarity_description', 'subjectivity_description', 
]
numeric_features = df[numeric_feature_names]
numeric_features.head()

,polarity_title,subjectivity_title,polarity_description,subjectivity_description
0,-0.004167,0.111111,0.106149,0.404040
1,0.000000,0.000000,0.000000,0.000000
2,-0.050000,0.100000,0.010683,0.398029
3,-0.025000,0.075000,0.087066,0.417113
4,0.000000,0.000000,0.000000,0.000000


In [10]:
tf.convert_to_tensor(numeric_features)

2023-05-19 18:31:02.046127: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.Tensor: shape=(465, 4), dtype=float64, numpy=
array([[-0.00416667,  0.11111111,  0.10614922,  0.40404026],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.05      ,  0.1       ,  0.01068309,  0.39802923],
       ...,
       [ 0.09444444,  0.15555556,  0.17327911,  0.4394066 ],
       [ 0.        ,  0.        ,  0.30833333,  0.55833333],
       [ 0.33214286,  0.70357143,  0.09209201,  0.3414066 ]])>

In [11]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [12]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_basic_model()
model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
233/233 [==============================] - 2s 5ms/step - loss: 0.6728 - accuracy: 0.6237
Epoch 2/15
233/233 [==============================] - 1s 4ms/step - loss: 0.6466 - accuracy: 0.6129
Epoch 3/15
233/233 [==============================] - 1s 4ms/step - loss: 0.6415 - accuracy: 0.6151
Epoch 4/15
233/233 [==============================] - 1s 4ms/step - loss: 0.6385 - accuracy: 0.6172
Epoch 5/15
233/233 [==============================] - 1s 5ms/step - loss: 0.6351 - accuracy: 0.6172
Epoch 6/15
233/233 [==============================] - 1s 3ms/step - loss: 0.6344 - accuracy: 0.6172
Epoch 7/15
233/233 [==============================] - 1s 4ms/step - loss: 0.6328 - accuracy: 0.6172
Epoch 8/15
233/233 [==============================] - 1s 3ms/step - loss: 0.6322 - accuracy: 0.6151
Epoch 9/15
233/233 [==============================] - 1s 3ms/step - loss: 0.6286 - accuracy: 0.6194
Epoch 10/15
233/233 [==============================] - 1s 4ms/step - loss: 0.6288 - accuracy: 0.6065

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 4)                9         
 n)                                                              
                                                                 
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 180
Trainable params: 171
Non-trainable params: 9
_________________________________________________________________


In [14]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [18]:
test_set = pd.read_csv("./final_results/test.csv")

In [21]:
numeric_feature_names = [
    'polarity_title', 'subjectivity_title',
    'polarity_description', 'subjectivity_description', 
]
numeric_features = test_set[numeric_feature_names]
tf.convert_to_tensor(numeric_features)
target = test_set.pop('target')
test_loss, test_acc = model.evaluate(x = numeric_features, y = target,verbose=2)
print('\nTest accuracy:', test_acc)

15/15 - 0s - loss: 0.6783 - accuracy: 0.6329 - 284ms/epoch - 19ms/step

Test accuracy: 0.6329113841056824
